# AIAA 2020 statstiscis table generator

Calculates the final statistics table laid out the paper (Table 1) 

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from pandas import Timestamp

import matplotlib.dates as mdates 

from mhdpy import *

import mhdpy


In [2]:
finalanalysisfolder = os.getcwd() #Folder with notebooks
dsst = mhdpy.load.loadprocesseddata(os.path.join(finalanalysisfolder,'Data', 'dsst'))

ds_p = xr.load_dataset(os.path.join(finalanalysisfolder, 'Data','spectral_data', 'fit_params.cdf'))
ds_p_stderr = xr.load_dataset(os.path.join(finalanalysisfolder,'Data', 'spectral_data', 'fit_params_stderr.cdf'))
ds_cantera = xr.load_dataset(os.path.join(finalanalysisfolder,'Data', 'ds_cantera.cdf'))

with open(os.path.join(finalanalysisfolder, 'Data','da_ct.pickle'), 'rb') as file:
    da_ct = pickle.load(file)
    
outputfolder = os.path.join(finalanalysisfolder, 'Data')


In [3]:
nK_expt = ds_p['nK_m3']
nK_expt.name ='nK_expt'
nK_expt_err = ds_p_stderr['nK_m3']

In [4]:
ds = analysis.ct.assign_tc_general(dsst['nhr'],da_ct)

da_mean, da_std = analysis.gen.bin_gen(ds, curname='I', voltname='V')

da_mean_highV = da_mean.sel(voltage=slice(50,100))#.drop(0.0, 'Kwt')

resist = da_mean_highV.coords['voltage']/da_mean_highV
resist = resist.mean('voltage', keep_attrs=True)
resist.attrs = dict(long_name = 'Resistance (50-100V) ', units = 'ohms')
resist = resist.where(resist>0)
resist = resist.where(resist<5000)
resist.name = 'resistance'

C:\Users\aspit\anaconda3\lib\site-packages\xarray\core\nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [5]:
ds_calor = ds_cantera[['wallheattransfer', 'chan_ht', 'comb_ht']]

#Need to assign test cases here as cantera is already set..
ds_inputs = xr.merge([dsst['hvof_input_calcs']['totalmassflow_hvof'], dsst['hvof_input_calcs']['Kwt_hvof']])
ds_inputs = analysis.ct.assign_tc_general(ds_inputs, da_ct)
ds_inputs = ds_inputs.mean('time', keep_attrs=True)

ds_cfd_input = xr.merge([dsst['hvof_input_calcs']['totalfuelflow_hvof'],dsst['hvof']['flow_o2_hvof'], dsst['syringe'][['em_massflow_seed','em_massflow_water']] ])
ds_cfd_input = analysis.ct.assign_tc_general(ds_cfd_input, da_ct)
ds_cfd_input = ds_cfd_input.mean('time', keep_attrs=True)

ds_stats = xr.merge([ds_inputs, resist,  nK_expt, ds_calor['chan_ht'], ds_calor['comb_ht'], ds_calor['wallheattransfer'], ds_cfd_input])

C:\Users\aspit\anaconda3\lib\site-packages\xarray\core\nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
C:\Users\aspit\anaconda3\lib\site-packages\xarray\core\nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


## Means table 

In [6]:
df_stats = ds_stats.mean('date').to_dataframe()
df_stats = df_stats.where(df_stats != 'nan').dropna()

df_stats.to_csv(os.path.join(outputfolder,'testcase_means.csv'))
df_stats

C:\Users\aspit\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


totalmassflow_hvof  Kwt_hvof   resistance       nK_expt   chan_ht  \
Kwt  tf                                                                         
0.10 9.52             9.515028  0.100929  1093.045341  6.952324e+20  5.401945   
     12.96           12.942129  0.100720  1009.415201  9.364073e+20  6.824548   
     16.38           16.382639  0.100134   996.160215  9.970920e+20  9.610468   
0.32 12.96           12.960052  0.317250   500.063503  3.114277e+21  6.566470   
1.00 12.96           12.958062  1.003572   439.674448  9.145815e+21  6.103389   

              comb_ht  wallheattransfer  totalfuelflow_hvof  flow_o2_hvof  \
Kwt  tf                                                                     
0.10 9.52   15.398073         20.800019            1.698753      7.729136   
     12.96  19.623602         26.448150            2.372167     10.451685   
     16.38  23.773086         33.383554            3.021395     13.212393   
0.32 12.96  19.388971         25.955441            2.328936     10.258040   
1.00 12.96  16.358089         22.461478            2.152614      9.625464   

            em_massflow_seed  em_massflow_water  
Kwt  tf                                          
0.10 9.52           0.016967           0.070172  
     12.96          0.023010           0.095163  
     16.38          0.028983           0.119869  
0.32 12.96          0.072649           0.300461  
1.00 12.96          0.229755           0.950215

## Statistics strings talbe with standard deviaitons

In [7]:
ds_stats_mean = ds_stats.mean('date', keep_attrs=True)
ds_stats_std = ds_stats.std('date', keep_attrs=True)

from xarray.plot.utils import label_from_attrs

#combine mean and std into one string

round_values = {
    'totalmassflow_hvof': 2,
    'resistance':1,
    'wallheattransfer': 1,
    'chan_ht': 2,
    'comb_ht': 2
}

da_statsstr = ds_stats_mean.where(False).astype(str)

seldicts = analysis.xr.gen_seldicts(ds_stats_mean)

for var in ds_stats_mean.data_vars: 
    for seldict in seldicts:
    
        m = ds_stats_mean[var].sel(seldict).item()
        s = ds_stats_std[var].sel(seldict).item()
        
        if var == 'nK_expt':
            m_str = '{:0.2e}'.format(m)
            s_str = '{:0.1e}'.format(s)
        elif var in round_values:
            m_str = str(round(m, round_values[var]))
            s_str = str(round(s, round_values[var]))            
        else:
            m_str = str(round(m, 3))
            s_str = str(round(s, 3))

        stat_str = m_str + ' +- ' + s_str

        if not m != m:
            da_statsstr[var].loc[seldict] = stat_str
    
    da_statsstr = da_statsstr.rename({var : label_from_attrs(ds_stats_mean[var])})

df_statsstr = da_statsstr.to_dataframe()
df_statsstr = df_statsstr.where(df_statsstr != 'nan').dropna()
df_statsstr.to_csv(os.path.join(outputfolder, 'testcase_stats_final.csv'))
df_statsstr

C:\Users\aspit\anaconda3\lib\site-packages\xarray\core\nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


Total Mass Flow [g/s]           K wt% Resistance (50-100V)  [ohms]  \
Kwt  tf                                                                         
0.10 9.52           9.52 +- 0.01    0.101 +- 0.0              1093.0 +- 209.8   
     12.96         12.94 +- 0.06  0.101 +- 0.001              1009.4 +- 159.2   
     16.38         16.38 +- 0.03      0.1 +- 0.0               996.2 +- 325.2   
0.32 12.96         12.96 +- 0.02    0.317 +- 0.0                500.1 +- 39.2   
1.00 12.96         12.96 +- 0.02  1.004 +- 0.002                439.7 +- 97.8   

           $n_{K,expt}$ [$\#/m^3$] Channel Heat Transfer [kW]  \
Kwt  tf                                                         
0.10 9.52      6.95e+20 +- 6.2e+19                5.4 +- 0.09   
     12.96     9.36e+20 +- 1.6e+20               6.82 +- 0.54   
     16.38     9.97e+20 +- 1.1e+20               9.61 +- 0.73   
0.32 12.96     3.11e+21 +- 4.4e+20               6.57 +- 0.42   
1.00 12.96     9.15e+21 +- 1.8e+20                6.1 +- 0.39   

           Combustor Heat Transfer [kW] Total Wall Heat Transfer [kW]  \
Kwt  tf                                                                 
0.10 9.52                  15.4 +- 0.64                   20.8 +- 0.7   
     12.96                19.62 +- 0.86                   26.4 +- 0.9   
     16.38                23.77 +- 0.55                   33.4 +- 0.9   
0.32 12.96                19.39 +- 0.78                   26.0 +- 0.4   
1.00 12.96                16.36 +- 0.87                   22.5 +- 1.3   

           Total Fuel Flow with Liquid\nSeed HVOF [g/s]  \
Kwt  tf                                                   
0.10 9.52                                1.699 +- 0.011   
     12.96                               2.372 +- 0.012   
     16.38                               3.021 +- 0.014   
0.32 12.96                               2.329 +- 0.007   
1.00 12.96                               2.153 +- 0.015   

           Mass Flow O2 HVOF [g/s] Emulsion K2CO3 Mass Flow [g/s]  \
Kwt  tf                                                             
0.10 9.52           7.729 +- 0.013                   0.017 +- 0.0   
     12.96         10.452 +- 0.054                   0.023 +- 0.0   
     16.38         13.212 +- 0.045                   0.029 +- 0.0   
0.32 12.96         10.258 +- 0.025                   0.073 +- 0.0   
1.00 12.96          9.625 +- 0.025                    0.23 +- 0.0   

           Emulsion Water Mass Flow [g/s]  
Kwt  tf                                    
0.10 9.52                     0.07 +- 0.0  
     12.96                   0.095 +- 0.0  
     16.38                    0.12 +- 0.0  
0.32 12.96                     0.3 +- 0.0  
1.00 12.96                    0.95 +- 0.0

# With CFD

In [9]:


ds_expt = xr.merge([resist,  nK_expt, ds_calor['chan_ht'], ds_calor['comb_ht']])
ds_expt = ds_expt.rename(nK_expt = 'nK')


ds_cfd = xr.load_dataset(os.path.join(finalanalysisfolder,'Data', 'ds_cfd.cdf'))
ds_cfd = ds_cfd.drop('T_outlet')
#CFD used nominal values, which sould be mentioned in the text. 
ds_cfd.coords['Kwt'] = ds_expt.coords['Kwt']
ds_cfd.coords['tf'] = ds_expt.coords['tf']


ds_cantera = xr.load_dataset(os.path.join(finalanalysisfolder,'Data', 'ds_cantera.cdf'))
ds_cantera= ds_cantera.rename(nK_cant = 'nK').rename(R_cantera = 'resistance').drop('T')

In [10]:
from xarray.plot.utils import label_from_attrs

#combine mean and std into one string

round_values = {
    'totalmassflow_hvof': 2,
    'resistance':1,
    'wallheattransfer': 1,
    'chan_ht': 2,
    'comb_ht': 2
}


ds_temp = ds_expt.mean('date')



# seldicts = analysis.xr.gen_seldicts(ds_stats_combine)

dss = []
for source in ['Expt', 'CFD', 'Cantera']:
    
    ds_statsstr = ds_temp.where(False).astype(str)
    seldicts = analysis.xr.gen_seldicts(ds_temp)
    
    for var in ds_temp.data_vars: 
        for seldict in seldicts:

            if source == 'CFD':
                stat_str = str(ds_cfd[var].sel(seldict).item())

            else:
                if source == 'Cantera':
                    m = ds_cantera.mean('date')[var].sel(seldict).item()
                    s = ds_cantera.std('date')[var].sel(seldict).item()
                else:
                    m = ds_expt.mean('date')[var].sel(seldict).item()
                    s = ds_expt.std('date')[var].sel(seldict).item()

                if var == 'nK':
                    m_str = '{:0.2e}'.format(m)
                    s_str = '{:0.1e}'.format(s)
                elif var in round_values:
                    m_str = str(round(m, round_values[var]))
                    s_str = str(round(s, round_values[var]))            
                else:
                    m_str = str(round(m, 3))
                    s_str = str(round(s, 3))



                if m != m:
                    stat_str = None
                else:
                    stat_str = m_str + ' +- ' + s_str

            if stat_str is not None:
                ds_statsstr[var].loc[seldict] = stat_str

    ds_statsstr = ds_statsstr.rename({var : label_from_attrs(ds_expt[var])})
    ds_statsstr = ds_statsstr.assign_coords(source=source).expand_dims('source')
    
    dss.append(ds_statsstr)

ds_statsstr = xr.merge(dss)
        
df_statsstr = ds_statsstr.to_dataframe()
df_statsstr = df_statsstr.swaplevel(1,2).sort_index(axis=0)
df_statsstr = df_statsstr.where(df_statsstr != 'nan').dropna()
df_statsstr.to_csv(os.path.join(outputfolder, 'testcase_stats_all.csv'))
df_statsstr

C:\Users\aspit\anaconda3\lib\site-packages\xarray\core\nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


resistance                   nK       chan_ht  \
Kwt  tf    source                                                        
0.10 9.52  CFD                574.7              8.7e+20           6.4   
           Cantera    255.4 +- 17.3  7.39e+20 +- 1.2e+19   5.4 +- 0.09   
           Expt     1093.0 +- 209.8  6.95e+20 +- 6.2e+19   5.4 +- 0.09   
     12.96 CFD                671.3             1.04e+21          8.54   
           Cantera    226.1 +- 12.5  7.61e+20 +- 9.9e+18  6.82 +- 0.54   
           Expt     1009.4 +- 159.2  9.36e+20 +- 1.6e+20  6.82 +- 0.54   
     16.38 CFD                786.2             1.18e+21         10.62   
           Cantera    225.4 +- 10.4  7.61e+20 +- 8.3e+18  9.61 +- 0.73   
           Expt      996.2 +- 325.2  9.97e+20 +- 1.1e+20  9.61 +- 0.73   
0.32 12.96 CFD                233.4             3.24e+21          8.99   
           Cantera     152.5 +- 3.8  2.34e+21 +- 1.6e+19  6.57 +- 0.42   
           Expt       500.1 +- 39.2  3.11e+21 +- 4.4e+20  6.57 +- 0.42   
1.00 12.96 CFD                173.5             9.25e+21          9.13   
           Cantera    138.0 +- 13.4  6.46e+21 +- 1.9e+20   6.1 +- 0.39   
           Expt       439.7 +- 97.8  9.15e+21 +- 1.8e+20   6.1 +- 0.39   

                   Combustor Heat Transfer [kW]  
Kwt  tf    source                                
0.10 9.52  CFD                             7.92  
           Cantera                 15.4 +- 0.64  
           Expt                    15.4 +- 0.64  
     12.96 CFD                             9.88  
           Cantera                19.62 +- 0.86  
           Expt                   19.62 +- 0.86  
     16.38 CFD                            11.63  
           Cantera                23.77 +- 0.55  
           Expt                   23.77 +- 0.55  
0.32 12.96 CFD                            10.37  
           Cantera                19.39 +- 0.78  
           Expt                   19.39 +- 0.78  
1.00 12.96 CFD                             9.93  
           Cantera                16.36 +- 0.87  
           Expt                   16.36 +- 0.87